Functional programming means thinking in terms of mathematical functions. Strict functional programming languages, such as Haskell, provide abstractions for functions which a very similar to their mathematical counterparts. However, most programming languages do not do this so well. Does this mean that the idea of modelling our abstractions in terms of mathematical functions is inappropriate when using these languages ?

A basic premise of this course is that, at least for many modern programming languages, this is not the case. The key to making this approach work is to understand the differences between a mathematical function and a function within the language being used, and then find effective ways of working around these differences.  

What then are the minimal requirements of a programming language for it to be able to support the use functional patterns ? The primary requirement is support for **Higher Order Programming**.



## Higher Order Programming

Higher Order Programming perhaps sounds more impressive than it actually is. Put simply it means that functions and other "higher order" types can be passed to and returned from functions in the same way as more traditional data e.g. floating point numbers and strings. 

### Example (python)

In [3]:
from math import sin,pi

def g(f,x) :
    return f(x)

g(sin,pi/6)

0.49999999999999994

### Example (R)

```R
g <- function(f,x)
{
    return(f(x))
}

g(sin(pi/6))
```

### Example (python)

In [6]:
def f(a) :
    def g(x) :
        return x + 2
    return g if a == 1 else sin

In [8]:
f(2)(5)

-0.9589242746631385

In [9]:
f(1)(5)

7

Notice that in the second example a new function **g** is defined inside an enclosing function **f**. When higher order programming is available most of the main functional design patterns can be implemented in some form. 

## Pattern 1 - Composition

### Exercise 

Write a function **compose** which accepts two functions **f** and **g** and returns a function **h** where $$h(x) = g(f(x))$$ 


## Pattern 2 - Currying

<figure>
<img src="./HaskellBCurry.jpg" style="height:20% display:inline-block">
<img src="./curry.jpeg" style="height:20% display:inline-block">
<figcaption align = "center"> Curry - a popular dish (left). Haskell B Curry - a computer scientist (right). </figcaption>
</figure>

In mathematics a function $f$ can be completely specified in terms of its graph $\Gamma_{f}$. The graph is a (possibly infinite) set of ordered pairs, for example

$\Gamma_{f} = \left\{  (1,1),(2,4),(3,9) \right\}$

Conceptually the graph shows how to map the first element of a given pair to the second. This is emphasised through the notation $f(1) = 1$ and $f(3) = 9$ and so on. 


### Exercise

Give a small example of a graph of function of the form $f(x,y)$.

Write a function in python that implements it.

How many "arguments" does the mathematical function "take" ? 

How many "arguments" does your python function "take" ?


So, mathematical functions map single value to single values. To make python functions behave in this way they can be **curried**. Here is an example.

In [10]:
from pymonad.tools import curry    

In [9]:
@curry(2)
def f(x,y) :
    return x + y

In [12]:
z = f(2,3)  # function taking two arguments
print(z)

5


In [13]:
g = f(2) # f is a function of one argument that returns a function 
z = g(3)
print(z)

5


In [14]:
z = f(2)(3)
print(z)

5


The "**curry**" pattern is slightly odd in that most functional programming languages do not need it since functions are curried by design.

### Exercise 

How can currying be put to practical use ?

### Solution

Currying allows a programmer to provide a way of adding extra parameters to a function whilst still allowing it to be **Reused** in other code. A strategy in the pig game provides an excellent example of a use case for the **curry** pattern. 

### Exercise

Try and use **curried** strategies in your pig competition code.

## Pattern 3 - Partial Application

Closely related to **Currying**, partial application accepts a function, freezes one or more of its arguments , and returns a new function with a reduced number of arguments. The frozen arguments are often referred to as the "bound" arguments or "bound" values.

In [15]:
def f(a,b,c) :
    return (2**a)*(3**b)*(5**c)

2700

In [17]:
f(2,3,2)

2700

In [19]:
from functools import partial

g = partial(f,b=3)
g(a=2,c=2)

2700

### Exercise

How does partial application differ from Currying ?

What effect does the position of the "bound" arguments have on the way the partially applied function is used ?

## Pattern 4 - Classes and Objects

### Motivating Problem

Here is a simple function

In [10]:
def f(x) :
    return 2*x

Now imagine you want to keep track of how many times this function gets used in a big program.

### Solution

In [13]:
count = 0

for i in range(10) :
    f(i)
    count = count + 1
# ... lots more code
# ...

# somewhere else in your program
y = f(8)
count = count + 1

# ... lots more code
# ...

# somewhere else in your program
z = f(3)/2
count += 1

# ... lots more code
# ...

### and finally
print("f was called " + str(count) + " times")

f was called 12 times


### Exercise 

What are the potential problems with this solution ?

Broadly speaking, how would you score this solution with respect to the **5Rs** ? 

### Exercise 

Design an alternative approach counting the number of times **f** gets used. 

Does your approach have any advantages over the original solution ? 

How would you rate your solution with respect to the **5Rs**

### Exercise (python)

Consider the following function

In [21]:
def logger(f) :
    count = 0
    def g(x) :
        count += 1
        return f(x),count
    return g

and the code below.

In [ ]:
def f(x) :
    return 2*x

logf = logger(f)

Describe what the function **logf** does. Check that it does what you expect.

### Closures, Classes, and Objects

The function **logf** is called a **closure**. It is a function. However, it is a function with extra information. The extra information is provided by the enclosing scope of the function (**f** in this case) within which the (nested) function **g** is defined. 

In general, a function that returns other functions is called a **class**, and the functions returned are called **objects**. If the **objects** have access to the data inside the scope of the **class** then they are **closures**. Note that the nested function is **not** the closure, it is the object that corresponds to the function. 

The programming style that makes use of **classes** and **objects** is called **object based programming**. 

### Exercise

How does using the **logger** class rate as a solution to the motivating problem ? 

What problem does using the **logger** class present ? 

Can you modify the **logger** class to overcome this problem ?

Think of, and describe, some use cases for **object based** programming. 

### Possible solution

In [16]:
def logger(f) :
    count = 0
    def call(x) :
        nonlocal count 
        count += 1
        return f(x)
    def log() :
        nonlocal count
        return count
    return call,log

In [20]:
call,log = logger(f)

for i in range(10) :
    call(i)  ## function gets used in the same way as the original

log()

10

## Pattern 5 - Memoisation



Memoisation is an example of an **object based** pattern. Here is way it might be implemented in python.

In [4]:
def memoise(f) :
    cache=dict()
    def mf(x) :
        nonlocal cache
        if not x in cache :
            cache[x] = f(x)
        return cache[x]
    return mf

### Exercise

Can you work out what this pattern is designed to do ?

Experiment with the following code to see what **memoisation** does.

In [2]:
def f(x) :
    print("calling f")
    return 2*x
y = f(3)
print(y)
y = f(3)
print(y)


calling f
6
calling f
6


In [5]:
mem_f = memoise(f)
y = mem_f(3)
print(y)
y = mem_f(3)
print(y)

calling f
6
6


### Exercise

Think of, and describe, some use cases for **memoisation**. 

### Exercise

If you can, have a go at implementing **memoisation** in another language. 

### Exercise

Are there any obvious problems with the **memoise** function presented above ?

### A more robust implementation of memoisation

In [22]:
from functools import cache

@cache
def g(x,y) :
    print("adding !!")
    return x +y

g(1,2)
g(3,4)
g(1,2)


adding !!
adding !!


3